In [1]:
import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg19
from tensorflow_vgg import utils

import os
from skimage import io, transform
import matplotlib.pyplot as plt

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

plt.ion() # interactive mode

In [31]:
def create_local_patches(layer, patch_size=1, padding='VALID'):
    return tf.extract_image_patches(layer, ksizes=[1, patch_size, patch_size, 1],
                                    strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding=padding)

In [3]:
def sort_images_by_landmarks(input_image, style_images, input_image_landmark=None, style_images_landmarks=None, n_best=16):
    """
    input_image_landmarks := landmarks of a training image (1 image)
    style_images_landmarks := landmarks of style images (60 images)
    """
    eucidean_distances = map_fn(lambda x: tf.losses.mean_squared_error(x, input_image_landmark)  )
    
    elems = np.array([1, 2, 3, 4, 5, 6])
    squares = map_fn(lambda x: x * x, elems, dtype=tf.float32)
    # squares == [1, 4, 9, 16, 25, 36]
    
    
    return style_images[:n_best]

In [4]:
graph = tf.Graph()
with graph.as_default():
    sess = tf.Session()
    
    # set vgg => will be normalized?
    vgg = vgg19.Vgg19()

/Users/user/Desktop/FastFaceSwapStyleLoss/tensorflow_vgg/vgg19.npy
npy file loaded


In [47]:
# set style image feed
style_images = []
for style_image_filename in os.listdir("style_data")[1:]:
    style_images.append(io.imread('./style_data/' + style_image_filename))
style_images_init = np.asarray(style_images)
print(style_images_init.shape)

(60, 160, 160, 3)


In [68]:
with graph.as_default():
    with sess.as_default():
        # input
        batch, height, width, channel = None, 160, 160, 3
        style_images_num = 60
        
        x = tf.placeholder(tf.float32, [batch, height, width, channel])
        fake_generated_image = tf.placeholder(tf.float32, [1, height, width, channel])
        style_images = tf.placeholder(tf.float32, [style_images_num, height, width, channel])
        sorted_style_images = sort_images_by_landmarks(x, style_images) # (16, 160, 160, 3)

        x_init = np.expand_dims(io.imread('./data/61192.jpg'), axis=0)
        fake_generated_image_init = np.expand_dims(io.imread('./data/61284.jpg'), axis=0)
        feed_dict = {
            x: x_init, 
            fake_generated_image: fake_generated_image_init,
            style_images: style_images_init,
        }

        vgg.build(fake_generated_image)
        relu3_1_fake_generated_image = vgg.conv3_1 # (1, 40, 40, 256)
        relu4_1_fake_generated_image = vgg.conv4_1
        
        vgg.build(sorted_style_images)
        relu3_1_sorted_style_images = vgg.conv3_1 # (16, 40, 40, 256)
        relu4_1_sorted_style_images = vgg.conv4_1

        
        
        
        # just relu3_1
        fake_generated_layer_patches = create_local_patches(relu3_1_fake_generated_image)
        style_images_layer_patches = create_local_patches(relu3_1_sorted_style_images)
        
        g_patches, s_patches = sess.run(
            [fake_generated_layer_patches, style_images_layer_patches],
            feed_dict=feed_dict)
        
#         _relu3_1_fake_generated_image, _sorted_style_images = sess.run(
#             [relu3_1_fake_generated_image, relu3_1_sorted_style_images], feed_dict=feed_dict)

build model started
build model finished: 0s
build model started
build model finished: 0s


In [69]:
g_patches.shape

(1, 40, 40, 256)

In [70]:
s_patches.shape

(16, 40, 40, 256)

In [73]:
g_patches.flatten()

array([ 125828.0078125 ,    4749.07617188,       0.        , ...,
             0.        ,   34419.46484375,   18869.73242188], dtype=float32)

In [76]:
tf.unstack(style_images_layer_patches)

[<tf.Tensor 'unstack_2:0' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:1' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:2' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:3' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:4' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:5' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:6' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:7' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:8' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:9' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:10' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:11' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:12' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:13' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:14' shape=(40, 40, 256) dtype=float32>,
 <tf.Tensor 'unstack_2:15' shape=(40, 40, 256) dtype=float32>]

In [ ]:
def patch_matching(generated_layer_patches, style_layers_patches, patch_size=1):
    for style_layer_patches in style_layers_patches:
        
        

In [ ]:
def mrf_loss(style_layers, generated_layer, patch_size=1, name=''):
    